In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split

import os
import shutil
import time
import matplotlib.pyplot as plt
import math
import pickle

from dataset import prepare_dataset
from dataset import ToFloatTensorInZeroOne
from model import LSTM_with_EFFICIENTNET

In [2]:
def get_time() -> str:
    return time.strftime('%c', time.localtime(time.time()))

def clear_pycache(root: str) -> None:
    if os.path.exists(os.path.join(root, '__pycache__')):
        shutil.rmtree(os.path.join(root, '__pycache__'))

def clear_log_folders(root: str) -> None:
    if os.path.exists(os.path.join(root, 'checkpoints')):
        shutil.rmtree(os.path.join(root, 'checkpoints'))
    if os.path.exists(os.path.join(root, 'history')):
        shutil.rmtree(os.path.join(root, 'history'))
    if os.path.exists(os.path.join(root, 'results')):
        shutil.rmtree(os.path.join(root, 'results'))

# For updating learning rate
def update_learning_rate(optimizer, lr) -> None:
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [4]:
# Set True when using Google Colab
colab = False

    # Consider GPU memory limit
    # Paper suggested 512
    # Suggest 128 for GTX 1660 Ti
batch_size = 16

    # Last checkpoint's training position
done_epochs = 0

    # Consider Google Colab time limit
    # How much epochs to train now
train_epochs = 0
 
clear_log = False
prepare_dataset(colab)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if colab:
    root = '/content/drive/MyDrive'
else:
    root = './'

if clear_log:
    clear_log_folders(root)

######## Preparing Dataset ########
print(f"Dataset | Data preparation start @ {get_time()}", flush=True)

timestamp = get_time().replace(':', '')
# timestamp = 'Tue Feb 14 191746 2023'

location = {
    'video_path': os.path.join(root, '../datasets/hmdb51dataset/video'),
    'annotation_path': os.path.join(root, '../datasets/hmdb51dataset/annotation'),
    'checkpoints_path': os.path.join(root, 'checkpoints', timestamp),
    'history_path': os.path.join(root, 'history', timestamp),
    'results_path': os.path.join(root, 'results', timestamp)
}
if not os.path.exists(location['checkpoints_path']):
    os.makedirs(location['checkpoints_path'])

if not os.path.exists(location['history_path']):
    os.makedirs(location['history_path'])

if not os.path.exists(location['results_path']):
    os.makedirs(location['results_path'])

# Preprocessing dataset
transform_train = transforms.Compose([
    ToFloatTensorInZeroOne(),
    transforms.Resize([256,342]),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomCrop(224)
])
transform_test = transforms.Compose([
    ToFloatTensorInZeroOne(),
    transforms.Resize([256,342]),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.CenterCrop(224)
])

dataset_train_val = torchvision.datasets.HMDB51(
    root=location['video_path'],
    annotation_path=location['annotation_path'],
    frames_per_clip=10,
    step_between_clips=5,
    train=True,
    transform=transform_train
)
dataset_test = torchvision.datasets.HMDB51(
    root=location['video_path'],
    annotation_path=location['annotation_path'],
    frames_per_clip=10,
    step_between_clips=5,
    train=False,
    transform=transform_test
)

# Train set 52.5%, validation set 17.5%, test set 30%
dataset_len = len(dataset_train_val)
train_len = math.floor(dataset_len * 0.75)
val_len = dataset_len - train_len
dataset_train, dataset_val = random_split(dataset_train_val, [train_len, val_len])

# Loading dataset
loader_train = DataLoader(
    dataset=dataset_train,
    batch_size=batch_size,
    shuffle=True,
    drop_last=False
)
loader_val = DataLoader(
    dataset=dataset_val,
    batch_size=batch_size,
    shuffle=True,
    drop_last=False
)
loader_test = DataLoader(
    dataset=dataset_test,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False
)

train_batches = len(loader_train)
val_batches = len(loader_val)
test_batches = len(loader_test)

Dataset | Data preparation start @ Sat Feb 25 03:24:37 2023


FileNotFoundError: [Errno 2] No such file or directory: './dataset/video'